In [1]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.utils import to_categorical, plot_model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:

features = {}
directory = os.path.join('/content/data', 'Images')

for image in tqdm(os.listdir(directory)):
    img_path = directory + '/' + image
    im = load_img(img_path, target_size=(224, 224))
    im = img_to_array(im)
    im = im.reshape((1, im.shape[0], im.shape[1], im.shape[2]))
    im = preprocess_input(im)
    feature = model.predict(im, verbose=0)
    im_id = image.split('.')[0]
    features[im_id] = feature

In [ ]:
pickle.dump(features, open(os.path.join('/content/Data', 'features.pkl'), 'wb'))

In [ ]:
with open(os.path.join('/content/Data', 'features.pkl'), 'rb') as f:
    features = pickle.load(f)

In [ ]:
with open(os.path.join('/content/Data/captions.txt'), 'r') as f:
    next(f)
    captions_doc = f.read()

In [ ]:
mapping = {}
for l in tqdm(captions_doc.split('\n')):
    count = l.split(',')
    if len(l) < 2:
        continue
    im_id, caption = count[0], count[1:]
    im_id = im_id.split('.')[0]
    caption = " ".join(caption)
    if im_id not in mapping:
        mapping[im_id] = []
    mapping[im_id].append(caption)

In [ ]:
def clean(mapping):
    for key, title in mapping.items():
        for i in range(len(title)):
            cptn = title[i]
            cptn = cptn.lower()
            cptn = cptn.replace('[^A-Za-z]', '')
            cptn = cptn.replace('\s+', ' ')
            cptn = 'startseq ' + " ".join([word for word in cptn.split() if len(word)>1]) + ' endseq'
            title[i] = cptn

In [ ]:
captions = []
for i in mapping:
    for cptn in mapping[i]:
        captions.append(cptn)

In [ ]:
tk = Tokenizer()
tk.fit_on_texts(captions)
vocab_size = len(tk.word_index) + 1

In [ ]:
max_length = max(len(cptn.split()) for cptn in captions)
max_length

In [ ]:
im_ids = list(mapping.keys())
split = int(len(im_ids) * 0.90)
test = im_ids[split:]
train = im_ids[:split]

In [ ]:

def data_generator(dk, mapping, feats, tk, max_length, vocab_size, b_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in dk:
            n += 1
            captions = mapping[key]
            for cptn in captions:
                seq = tk.texts_to_sequences([cptn])[0]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    X1.append(feats[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == b_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield {"image": X1, "text": X2}, y
                X1, X2, y = list(), list(), list()
                n = 0

In [ ]:
I1 = Input(shape=(4096,), name="image")
fv1 = Dropout(0.4)(I1)
fv2 = Dense(256, activation='relu')(fv1)

I2 = Input(shape=(max_length,), name="text")
s1 = Embedding(vocab_size, 256, mask_zero=True)(I2)
s2 = Dropout(0.4)(s1)
s3 = LSTM(256)(s2)

decodera = add([fv2, s3])
decoderb = Dense(256, activation='relu')(decodera)
outputs = Dense(vocab_size, activation='softmax')(decoderb)

model = Model(inputs=[I1, I2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

plot_model(model, show_shapes=True)

In [ ]:
epochs = 20
b_size = 32
steps = len(train) // b_size

for i in range(epochs):
    generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, b_size)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

In [ ]:
model.save(WORKING_DIR+'/best_model.h5')

In [ ]:
def idx_to_word(i, tk):
    for word, c in tk.word_index.items():
        if c == i:
            return word
    return None

In [ ]:
def predict_caption(model, im, tk, max_length):
    t = 'startseq'
    for i in range(max_length):
        sequence = tk.texts_to_sequences([t])[0]
        sequence = pad_sequences([sequence], max_length)
        x = model.predict([im, sequence], verbose=0)
        x = np.argmax(x)
        word = idx_to_word(x, tk)
        if word is None:
            break
        t += " " + word
        if word == 'endseq':
            break
    return t

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
actual, predicted = list(), list()

for t in tqdm(test):
    lst = mapping[t]
    y_pred = predict_caption(model, features[t], tokenizer, max_length)
    actual_captions = [caption.split() for cptn in lst]
    y_pred = y_pred.split()
    actual.append(actual_captions)
    predicted.append(y_pred)

print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
def generate_caption(image_name):
    # load the image
    # image_name = "1001773457_577c3a7d70.jpg"
    image_id = image_name.split('.')[0]
    img_path = os.path.join(BASE_DIR, "Images", image_name)
    image = Image.open(img_path)
    captions = mapping[image_id]
    print('---------------------Actual---------------------')
    for caption in captions:
        print(caption)
    # predict the caption
    y_pred = predict_caption(model, features[image_id], tokenizer, max_length)
    print('--------------------Predicted--------------------')
    print(y_pred)
    plt.imshow(image)